# 4-3: Check qartod test variables for tests executed

2025-06-13

The QARTOD `_qartod_executed` variables for some parameters have been missing the `tests_executed` attribute that lists as a string which tests were run and indicates which of the results in each string element correspond to specific test results.

Variables with the observed issue:
<ul>
    <li>GA01SUMO-RII11-02-PCO2WC051/pco2w_abc_imodem_instrument_recovered/pco2_seawater</li>
    <li>GS01SUMO-SBD11-08-NUTNRB000/nutnr_b_dcl_dark_full_instrument_recovered/nitrate_concentration</li>
</ul>

Slightly different observed abnormality, where the lookup tables exist and tests are in production (?), but no tests were executed.
<ul>
    <li>CP10CNSM-MFD37-03-CTDBPC000/ctdbp_cdef_dcl_instrument/pressure, temp, practical_salinity</li>
</ul>

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import xarray as xr
import requests
import io
import ast
from glob import glob

In [2]:
# Import functions from ooinet and ooi_data_explorations libraries
from ooi_data_explorations.common import load_kdata, get_vocabulary, m2m_request, m2m_collect
from ooinet import M2M

In [3]:
# Import functions from project qc_completion module
from qartod_testing.qc_completion import load_gross_range_qartod_test_list, \
    load_climatology_qartod_test_list, make_test_parameter_dict, \
    check_tests_exe, make_results_table, add_test_exe, write_results

### GA01SUMO-RII11-02-PCO2WC051
Affected stream: pco2w_abc_imodem_instrument_recovered 

Affected variables: pco2_seawater

In [7]:
refdes = "GA01SUMO-RII11-02-PCO2WC051"
site, node, sensor = refdes.split("-", 2)
method = "recovered_host"
stream = "pco2w_abc_imodem_instrument_recovered"
deploy = 1
instclass = sensor[3:8]
data = load_kdata(site, node, sensor, method,
                      stream, ('*deployment%04d*%s*.nc' % (deploy, instclass))) # no regex
if data is None:
    m2m_result = m2m_request(site, node, sensor, method, stream)
    data = m2m_collect(m2m_result, tag=('.*deployment%04d.*%s.*.nc$' % (deploy, instclass))) # yes regex

Loading and Processing Data Files: 100%|██████████| 2/2 [00:00<00:00, 30.20it/s]


In [6]:
data

<xarray.Dataset>
Dimensions:                                   (time: 1571, spectrum: 14)
Coordinates:
  * spectrum                                  (spectrum) int32 0 1 2 ... 12 13
  * time                                      (time) datetime64[ns] 2015-03-1...
Data variables: (12/26)
    pco2w_thermistor_temperature_qc_executed  (time) uint8 1 1 1 1 1 ... 1 1 1 1
    pco2_seawater_qartod_results              (time) float32 1.0 1.0 ... 255.0
    pco2_seawater_qc_executed                 (time) float32 29.0 29.0 ... 29.0
    absorbance_ratio_434_qc_executed          (time) float32 9.0 9.0 ... 9.0 9.0
    absorbance_blank_620                      (time) float64 2.412e+04 ... 2....
    record_type                               (time) uint8 4 4 4 4 4 ... 4 4 4 4
    ...                                        ...
    absorbance_ratio_620_qc_executed          (time) float32 9.0 9.0 ... 9.0 9.0
    record_time                               (time) uint32 3509301609 ... 35...
    absorbance_ratio_620_qc_results           (time) float32 9.0 9.0 ... 9.0 9.0
    thermistor_raw                            (time) int16 2013 2010 ... 2153
    pco2_seawater                             (time) float64 337.3 ... 363.5
    unique_id                                 (time) uint8 122 122 ... 122 122
Attributes: (12/69)
    node:                               RII11
    comment:                            Data produced by the OOI M2M API and ...
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  recovered_host
    stream:                             pco2w_abc_imodem_instrument_recovered
    ...                                 ...
    geospatial_lon_resolution:          0.1
    geospatial_vertical_units:          meters
    geospatial_vertical_resolution:     0.1
    geospatial_vertical_positive:       down
    lat:                                -42.92283
    lon:                                -42.4393

Neither the `pco2_seawater_qartod_results` nor `pco2_seawater_qartod_executed` parameters have any attributes. This is what is making the test executed check fail. The last value in the QARTOD results is also not a valid QARTOD flag ("255"). The `qartod_executed` string for this data point is empty.

In [9]:
data["pco2_seawater_qartod_results"]

<xarray.DataArray 'pco2_seawater_qartod_results' (time: 1571)>
array([  1.,   1.,   1., ...,   1.,   1., 255.], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 2015-03-15T22:00:09 ... 2015-07-31T10:00:09

In [10]:
data["pco2_seawater_qartod_executed"]

<xarray.DataArray 'pco2_seawater_qartod_executed' (time: 1571)>
array(['11', '11', '11', ..., '11', '11', ''], dtype=object)
Coordinates:
  * time     (time) datetime64[ns] 2015-03-15T22:00:09 ... 2015-07-31T10:00:09

## GS01SUMO-SBD11-08-NUTNRB000
Affected stream: nutnr_b_dcl_dark_full_instrument_recovered

Affected variables: nitrate_concentration

## CP10CNSM-MFD37-03-CTDBPC000
Affected stream: ctdbp_cdef_dcl_instrument

Affected variables: pressure, temp, practical_salinity

After using get_vocabulary and changing to BASH not RegEx wildcards, data loaded correctly, and the qartod results were present. How did that happen?

In [5]:
refdes = "CP10CNSM-MFD37-03-CTDBPC000"
site, node, sensor = refdes.split("-", 2)
method = "telemetered"
stream = "ctdbp_cdef_dcl_instrument"
deploy = 1
instclass = sensor[3:8]
get_vocabulary(site, node, sensor)
data = load_kdata(site, node, sensor, method,
                      stream, ('*deployment%04d*%s*.nc' % (deploy, instclass)))

Loading and Processing Data Files: 100%|██████████| 1/1 [00:00<00:00,  7.91it/s]


In [6]:
data

<xarray.Dataset>
Dimensions:                                       (time: 2470, string5: 5)
Coordinates:
  * time                                          (time) datetime64[ns] 2024-...
Dimensions without coordinates: string5
Data variables: (12/16)
    sea_water_pressure                            (time) float32 29.67 ... 30.52
    dcl_controller_timestamp                      (time, string5) |S1 b'e' .....
    sea_water_temperature_qartod_results          (time) uint8 1 1 1 1 ... 1 1 1
    internal_timestamp                            (time) datetime64[ns] 2024-...
    sea_water_pressure_qartod_executed            (time) object '1' '1' ... '1'
    date_time_string                              (time, string5) |S1 b'e' .....
    ...                                            ...
    sea_water_temperature                         (time) float32 9.209 ... 15.06
    sea_water_density                             (time) float64 1.025e+03 .....
    depth                                         (time) float64 29.45 ... 30.29
    sea_water_pressure_qartod_results             (time) uint8 1 1 1 1 ... 1 1 1
    sea_water_electrical_conductivity             (time) float32 3.47 ... 3.893
    sea_water_practical_salinity_qartod_executed  (time) object '1' '1' ... '1'
Attributes: (12/69)
    node:                               MFD37
    comment:                            Data produced by the OOI M2M API and ...
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  telemetered
    stream:                             ctdbp_cdef_dcl_instrument
    ...                                 ...
    geospatial_lon_resolution:          0.1
    geospatial_vertical_units:          meters
    geospatial_vertical_resolution:     0.1
    geospatial_vertical_positive:       down
    lat:                                35.94988
    lon:                                -75.11943